--------------------------------------
# *Docker cheatsheet*
--------------------------------------

# General ``docker`` (CLI) commands

There is no need to remind that, for every of the **following commands**, more details can be found in the *built-in guide* if the use the **``--help``** flag 

### ``docker search <registry-address>``

This command provides a list of images (official or not) with similar registry name.

### ``docker pull <registry-address>:<tag>``

Used to pull the ``<registry>`` image from the Docker registry (at Dockerhub) and saves it to our system

Let's remind that a ``tag`` refers to a particular snapshot of the image, whose unique identifier is the ``image-id``. The ``tag`` can be specified with dots after the registry address (usually ``user/image-name`` if they are not official images) 

Most of times the registry address is something like:
```docker
registry/repo/name:tag
```
For instance,
```docker
docker.elastic.co/elasticsearch/elasticsearch:6.3.2
```

### ``docker push <image-name>``

Once we have built an image from a Dockerfile named ``<image-name>`` this can be pushed into the remote (Dockerhub) using the ``push`` command.

## *Images* commands

### ``docker build -t <image-name> <location-of-Dockerfile>``

Create an image with name ``<image-name>`` based on the Dockerfile found at ``<location-of-Dockerfile>``  

### ``docker images``

Provides a list of all images on your system

### ``docker image prune``

Remove dangling images. Note that images that you no longer need can be deleted by running:
```docker
docker rmi
```

## *Containers* commands

### ``docker run --rm -d -it --name <container-name> --net <network-name> -p <local-port>:<container-port> -v <host_directory>:<container_directory> -e "<var-name>=<var-value>" <image>``

Run a Docker container based on the passed image:
- The ``--name`` tag gives a name to the container which will be created
- The ``--net`` tag specifies inside which network this container will be launched
- The ``-d`` flag starts the container in detached mode, (the container runs in the background and doesn't keep the terminal occupied)
- The ``-it`` flag attaches us to an interactive *tty* (virtual terminal session)
- The ``-p`` flag allows us to specify the port at which the docker will hear
- _Bind mount_: the ``v`` flag tells Docker engine to **mount** the given host directory to the container directory
- The ``-e`` flag passes an environment variable to the container
- The ``-P`` flag tells Docker to publish all exposed ports of the container to random ports on the host interface (equivalent to using the ``-p`` flag with an empty value for the host port (-p :<container_port>). For example, if your Dockerfile exposes port 80 (``EXPOSE 80``), using -P will publish the container's port 80 to a random port on the host machine (allowing you to access the containerized application through that random port on your local machine).
- The ``--rm`` tag tells Docker to automatically remove the container and its associated file system after it exits (useful for ephemeral containers, such as those used for testing, because it ensures that any changes made to the container's file system are discarded once the container is stopped).

For instance, an example of query could be:

```docker
docker run -d --name es --net bridge -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" <image-name>
```
Being, for instance, ``<image-name>``=``docker.elastic.co/elasticsearch/elasticsearch:6.3.2`` this address of the Elasticsearch image 

### ``docker container logs <container-name>``

Checks logs on the container. We can also perform a curl to see if the passed port is listening:
```bash
curl 0.0.0.0:<local-port>
```

Bearing in mind 0.0.0.0 is the private IP address of the localhost

### ``docker ps -a -q -f <condition>``

List the running containers **history**:
- The ``-a`` flag is for more information
- The ``-q`` flag is to force the **``ps``** to return just the numeric IDs of the containers 
- The ``-f`` flag filters the output based on conditions provide: _e.g._ ``status=exited``

Note ``docker container ls`` is equivalent to ``docker ps``

### ``docker rm <container ids>``

Delete a/some container/s given its/their id/s

### ``docker container prune``

Deletes all containers that have a status of exited. Note the same can be achieved by doing:
```docker
$ docker rm $(docker ps -a -q -f status=exited)
```

## _Volumes_ commands

Good for sharing data among multiple running containers and you want decouple the configuration of the Docker host from the container runtime (or even store the container's data on a remote host or a cloud provider rather than locally).

The volume's directory can be always backed up even though the container's not running: ``/var/lib/docker/volumes/<volume-name>``.

### ``docker volume create``

Create a Docker volume

### ``docker volume prune``

When no running container is using a volume, the volume is still available to Docker (**not** automatically **removed**). If we want to remove unused volumes we have to use the instruction above

## _Networks_ commands

### ``docker network inspect <network-name>`` 

Where ``<network-name>`` is the network name. There 3 already existing by default:
- ``bridge``
- ``host``
- ``none``

Actually, typing ``docker network inspect bridge`` will tell us which is the private IP address through which the container can be accessed.

Even though this network is shared amongst container, the IP address can change. So how we tell to another container which is the hostname of one container? This can be done if we...

### Create a network: ``docker network create <network-name>``

And we can see the list of created networks with ``docker network ls``

Most importantly, to launch the containers inside this network we have to specify it with the ``--net <network-name>`` flag in the ``docker run`` command (e.g., ``docker run -d --name es --net foodtrucks-net``)

# Dockerfile cheatsheet

Below you can find a complete list of commands to use in a Dockerfile: https://docs.docker.com/engine/reference/builder/

The most common commands found in a Dockerfile are:
- ``FROM <registry-address>:<tag>``: specifying the base image (if the created image is a child image) 
- ``EXPOSE <port>``: set which ports of the containers are available from outside
- ``CMD [<command-1>, <command-2>]``: run a ``CMD`` command
- ``RUN <bash-command>``: run a ``bash`` command
- ``COPY <local-file> <container-directory>``: we copy the local file into the passed container directory
- ``ADD <local-folder> <volume-address>``: we copy our local folder into a new volume in the container (analogous to ``COPY`` but for local folders, which are set as volumes)
- ``WORKDIR <container-directory>``: we set the passed directory as the working directory

# Multi-container applications: ``docker-compose``

With Compose, you use a YAML file to 
configure your application’s services. Then, with a single 
command, you create and start all the services from your
configuration.
Using Compose is basically a three-step process:
1. Define your app’s environment with a Dockerfile so it can be reproduced anywhere.
2. Define the services that make up your app in docker-compose.yml so they can be run together in an isolated environment.
3. Run ``docker-compose`` up and Compose starts and runs your entire app

## Example of ``docker-compose.yml`` file

```yml
version: "3"
services:
  es:
    image: docker.elastic.co/elasticsearch/elasticsearch:6.3.2
    container_name: es
    environment:
      - discovery.type=single-node
    ports:
      - 9200:9200
    volumes:
      - esdata1:/usr/share/elasticsearch/data
  web:
    image: rohit1308k/foodtrucks-web
    command: python app.py
    depends_on:
      - es
    ports:
      - 5000:5000
    volumes:
      - ./flask-app:/opt/flask-app
volumes:
    esdata1:
      driver: local
```

There are 2 main levels we have to define:
- Services
- Volumes

There can be several services. After listing them by name, we can define what they do throgh diferent commands, such as: ``image``, ``container_name``, ``command``, ``depends_on``, ``environment``, ``ports``, ``volumes``...

## Known commands

For instance:
- ``docker-compose up -d``: ``-d`` for detached
- ``docker-compose down -v``: ``-v`` to delete the volumes